In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
from imdbUtils import *
import csv
pd.options.display.max_colwidth=500
from tqdm import tqdm

# Create Corona Dataset

In [ ]:
import os
all_tables = []

directory = 'data/Corona/tables/'
i = 0
for root, dirs, files in os.walk(directory):
    for file in files:
        with open(directory + file) as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            
            next(spamreader)
            for row in spamreader:
                temp = [file.split('.')[0].replace('_',' ')]
                temp.append(row[0].lower())
                for r in row[-6::]: temp.append(str(int(float(r))))
                all_tables.append(temp)


In [ ]:
table_columns = {1:'table',2:'country',3:'january',4:'february',5:'march',6:'april',7:'may',8:'june'}

In [ ]:
all_claims = {}

with open('data/Corona/Claims_English.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for r in csv_reader:
        if r[-1] == 'a':
            all_claims[r[0].lower()] = [r[1].replace('_',' '),r[2].lower(),r[3].lower()]

In [ ]:
import csv
with open('corona_GEN.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for claim in all_claims:
        temp = [claim]
        for c in all_claims[claim][0:4]: temp.append(c)
        spamwriter.writerow(temp)


In [ ]:
user_claims = {}

with open('data/Corona/Claims_users.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for r in csv_reader:
        user_claims[r[0].lower()] = []        
        for rr in r[1].split('-'):
            for rrr in r[2].split('-'):
                temp = [rr.lower(),rrr.replace('_',' ').lower()]
                for rrrr in r[3::]:
                    temp.append(rrrr.lower().replace('_',' '))
            user_claims[r[0].lower()].append(temp)


# Create Graph

In [ ]:
import utils
import graphUtils
import networkx as nx

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
import nltk
ps = nltk.stem.PorterStemmer()

def return_n_grams(text,k):
    tokens = word_tokenize(text)
    n_grams = set()
    for i in range(0,len(tokens)-(k-1)):
        n_grams.add( ' '.join( ( [tk for tk in tokens[i:i+k]]) ))
        
    return n_grams


def find_all_n_grams (text,n):
    n_grams = []
    for k in range(1,n+1):
        k_grams = return_n_grams(text,k)
        for g in k_grams: n_grams.append(g)
    return n_grams


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

G=nx.Graph()
K = 2

i = 0
nodes_labels = {}
row_ids = {}
id_rows = {}

for row in tqdm(all_tables):
    i+=1
    row_name = str('RW'+str(i))
    G.add_node(row_name , label= row_name, type='Row')
    row_ids[row_name] = ' '.join([r for r in row])
    id_rows[' '.join(row)] = row_name
    
    j=0
    for cl in row:
        j+=1
        col_name = table_columns[j]
                
        if not G.has_node(col_name):     G.add_node(col_name , label= col_name, type='Column')
        n_grams = [gr.replace(' ','_') for gr in find_all_n_grams(str(cl),K)]
        
        for tg in n_grams:
            if not G.has_node(tg): G.add_node(tg,label=tg, type='Token')
            G.add_edge(row_name,tg)
            G.add_edge(col_name,tg)
            
            

In [ ]:
i = 0
claim_ids = {}
id_claim = {}
all_claims.update(user_claims)

for claim in tqdm(all_claims):
    i += 1
    text = remove_stopwords(claim)
    claim_name = str('Claim'+str(i))
    G.add_node(claim_name , label= claim_name, type='Claim')
    claim_ids[claim_name] = claim
    id_claim[claim] = claim_name
    
    n_grams = [gr.replace(' ','_') for gr in find_all_n_grams(text,K)]
    
    for tg in n_grams:
        token = tg
            
        if not G.has_node(token): continue

        if not G.has_edge(claim_name,token):            G.add_edge(claim_name,token)


In [ ]:
ground_truth = {}

for cl in claim_ids:
    ground_truth[cl] = []
    if claim_ids[cl] in user_claims:
        for r in user_claims[claim_ids[cl]]:
            for rr in id_rows:
                if ' '.join(reversed(r)) in rr:             
                    ground_truth[cl].append(id_rows[rr])
    else:
        for r in id_rows:
            if ' '.join(all_claims[claim_ids[cl]][0:2]) in r:
                ground_truth[cl].append(id_rows[r])
                

# RandomWalk

In [ ]:
docs = []
random_paths = generate_random_walks(20,l=20)
for p in random_paths:
    docs.append(p)

# Word Embeddings

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from nltk.tokenize import word_tokenize

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from tqdm import tqdm 
tagged_data = []
for d in tqdm(docs,position=0):
    tagged_data.append(word_tokenize(d))


In [ ]:
%env PYTHONHASHSEED=0
max_epochs = 30
vec_size = 300

model = Word2Vec(size=vec_size, min_count=10, window=3, sg=1, seed=0, workers = 4)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

print("Model is Ready")

In [ ]:
movie_reviews = {}
for claim in tqdm(ground_truth):
    if claim in model.wv:
        movie_reviews[claim] = utils.distance_w2v (model,claim,row_ids,len(row_ids))

In [ ]:
for KK in [1,2,3,5,10,20,50]: 
#for KK in [2]: 
    
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for movie in movie_reviews:
        if movie not in ground_truth or len(ground_truth[movie])==0: continue
        #if movie in [id_claim[c] for c in user_claims]: continue
            
        i+=1
        preds = [f for (f,j) in movie_reviews[movie]][0:KK]
        golds = [g for g in ground_truth[movie]]
        
        MAP += utils.MAP_K(golds,preds)
        MRR += utils.MRR(golds,preds)
        hasP += utils.HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)